# JetBot - Data collection

In this notebook we'll collect training data for CNN VAE. The training data save to dataset directory. Need USB gamepad for running.

## Import module

Import required module for this notebook. MobileController module is own module. This module implements Jetbot steering control.


In [1]:
import os
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

## Activation Gamepad.
This step is similar to "Teleoperation" task. In this task, we will use gamepad controller to collect training data.

The first thing we want to do is create an instance of the Controller widget, which we'll use to control jetbot with speed and steering. The Controller widget takes a index parameter, which specifies the number of the controller. This is useful in case you have multiple controllers attached, or some gamepads appear as multiple controllers. To determine the index of the controller you're using,

Visit http://html5gamepad.com. Press buttons on the gamepad you're using Remember the index of the gamepad that is responding to the button presses Next, we'll create and display our controller using that index.


In [2]:
controller = widgets.Controller(index=0)
display(controller)

Controller()

Now we can look Button assignment of gamepad. let check your need key assignment.

* SPEED is left side joystick uptodown almost is joys 1
* STERING is right side joystick left-right almost is joys2
* RECORDING is right side  trigger almost is buttons 5


In [3]:
SPEED=1
STERING=2
RECORDING=5

## Steering Control

Steering control need to "wheel track" parameter. Wheel track is between distance left-right wheels.
We'll measure wheel track  measure in cm.

In [4]:
car = NvidiaRacecar(steering_channel=1, throttle_channel=0)
car.steering_gain = -0.65
car.throttle_gain = 0.3

We'll run next block. and We can controll jetbot with gamepad.

In [5]:
speed = widgets.FloatSlider(min=-1.0, max=1.0, description='speed')
steering = widgets.FloatSlider(min=-1.0, max=1.0, description='steering')


speed_link = traitlets.dlink((controller.axes[SPEED], 'value'), (car, 'throttle'), transform=lambda x: -x)
steering_link = traitlets.dlink((controller.axes[STERING], 'value'), (car, 'steering'))
traitlets.dlink((car, 'throttle'), (speed,'value'))
traitlets.dlink((car, 'steering'), (steering, 'value'))
display(widgets.HBox([speed,steering]))

## Initialize Camera

Next is initializing camera module. Image size is 320 x 240. Frame rate is about 27Hz. We'll save image in camera observer method. camera observer method can get image per frame rate. Thus, frame rate is decide to image save interval.

In [6]:
CAMERA_WIDTH = 320
CAMERA_HEIGHT = 240

camera = CSICamera(width=CAMERA_WIDTH, height=CAMERA_HEIGHT, capture_width=CAMERA_WIDTH, 
                   capture_height=CAMERA_HEIGHT, capture_fps=60)

camera.running = True

image = widgets.Image(format='jpeg', width=320, height=240)
camera_link = traitlets.dlink((camera,'value'), (image,'value'), transform=bgr8_to_jpeg)
#display(image)


# camera = Camera.instance(width=320, height=240)
# image = widgets.Image(format='jpeg', width=320, height=240)
# camera_link = traitlets.dlink((camera,'value'), (image,'value'), transform=bgr8_to_jpeg)


## UI Widget

We can check Gamepad value and Image. 

In [7]:
DATASET_DIR = 'dataset'
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

dataset=DATASET_DIR
layout = widgets.Layout(width='100px', height='64px')
count_box   = widgets.IntText(layout=layout, value=len(os.listdir(dataset)))
count_label = widgets.Label(layout=layout, value='Number image:')
count_panel = widgets.HBox([count_label,count_box])

panel = widgets.VBox([count_panel, speed, steering])
display(widgets.HBox([panel,image]))


Directories not created becasue they already exist


## Set callback for collect the training data.

```save_record``` is callback for training data. The method set to camera observer. This callback saving the image that contain speed and steering in file name to DATASET_DIR. When holding ```R``` button, this method recording training data. You can check number of training data with ```Number image text box```.

In [8]:

import os
from uuid import uuid1

def save_record(change):                
    global controller, car, speed, steering
    if controller.buttons[RECORDING].value==1.0:
        
        image_name = '{:.02f}_{:.02f}_{}.jpg'.format(car.throttle, car.steering,uuid1())
        image_path = os.path.join(DATASET_DIR, image_name)
        save_image=bgr8_to_jpeg(change['new'])
        with open(image_path, 'wb') as f:
            f.write(save_image)
        count_box.value = len(os.listdir(dataset)) 


save_record({'new': camera.value})
camera.observe(save_record, names='value')

## Cleanup
After collecting enough data. cleanup camera observer and stop all motor.

In [9]:
camera.unobserve_all()
speed_link.unlink()
steering_link.unlink()


## Cleate dataset.zip file

In [ ]:
import datetime
def timestr():
    return str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

!zip -r -q jetbot_{DATASET_DIR}_{timestr()}.zip {DATASET_DIR}